# LongChain: Memory

## Outline
* ConversationBufferMemory
* ConversionBufferWindowsMemory
* ConversionTokenBufferMemory
* ConversationSummaryMemory

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Account for deprecation LLM model
import datetime
current_date = datetime.datetime.now().date()  # Get the current date

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## ConversationBufferMemory

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain (
    llm = llm,
    memory = memory,
    # verbose = False
    verbose = True  # To see what langchain is actually doing
)

In [ ]:
conversation.predict(input="Hi, my name is Hassane")

In [ ]:
conversation.predict(input="What is 1+1")

In [ ]:
conversation.predict(input="What is my name?")

As we can in the above cells, the conversation history/memory is getting longer and longer

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
# Add new things to the memory
memory.save_context({"input": "Hi"}, {"outputs": "What\'s up"})

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "Not much, just hangin"}, {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

## ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)

In [ ]:
memory.save_context({"input": "Hi"}, {"outputs": "What\'s up"})
memory.save_context({"input": "Not much, just hangin"}, {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain (
    llm = llm,
    memory = memory,
    verbose = True
)

In [ ]:
conversation.predict(input="Hi, my name is Hassane")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")